# Training

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # Possible GPUs

In [ ]:
import torch
import sys; sys.path.insert(0, '..')
import mair

### Settings

In [ ]:
PATH = "./models/"
NAME = "Sample"
SAVE_PATH = PATH + NAME

MODEL_NAME = "ResNet18"
DATA = "CIFAR10"
MEAN = [0.4914, 0.4822, 0.4465]
STD = [0.2023, 0.1994, 0.2010]

N_VALIDATION = 1000
N_CLASSES = 10

EPOCH = 200

EPS = 8/255
ALPHA = 2/255
STEPS = 10

### Load Data

In [ ]:
import torchvision.datasets as dsets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), 
                         (0.2023, 0.1994, 0.2010))
])

MEAN = [0.4914, 0.4822, 0.4465]
STD = [0.2023, 0.1994, 0.2010]

train_data = dsets.CIFAR10(root='./data', 
                           train=True,
                           download=True, 
                           transform=transform)

test_data  = dsets.CIFAR10(root='./data', 
                           train=False,
                           download=True, 
                           transform=transform)

batch_size = 128
train_loader = DataLoader(train_data, 
                          batch_size=batch_size,
                          shuffle=True)

test_loader = DataLoader(test_data, 
                         batch_size=batch_size,
                         shuffle=False)

### Load Model

In [ ]:
model = mair.utils.load_model(model_name=MODEL_NAME, 
                              n_classes=N_CLASSES).cuda() # Load model
rmodel = mair.RobModel(model, n_classes=N_CLASSES, 
                       normalization_used={'mean':MEAN, 'std':STD}).cuda()

# For multi-GPUs
# rmodel = rmodel.set_parallel() 

### Start Training

In [ ]:
from mair.defenses import AT
trainer = AT(rmodel, eps=EPS, alpha=ALPHA, steps=STEPS)
trainer.record_rob(train_loader, test_loader, eps=EPS, alpha=2/255, steps=10, std=0.1, 
                   n_train_limit=N_VALIDATION, n_val_limit=N_VALIDATION)

In [ ]:
trainer.setup(optimizer="SGD(lr=0.1, momentum=0.9, weight_decay=0.0005)",
              scheduler="Step(milestones=[100, 150], gamma=0.1)", 
              scheduler_type="Epoch",
              minimizer=None,
              n_epochs=EPOCH, n_iters=len(train_loader)
             )

In [ ]:
trainer.fit(train_loader=train_loader, n_epochs=EPOCH,
            save_path=SAVE_PATH, save_best={"Clean(Val)":"HBO", "PGD(Val)":"HB"},
            save_type=None, save_overwrite=True, record_type="Epoch")

### Evaluation

In [ ]:
rmodel.load_dict('./models/CIFAR10_ResNet18_AT(eps=8, alpha=2, steps=10)/best.pth')

In [ ]:
rmodel.eval_accuracy(test_loader)  # clean accuracy

In [ ]:
rmodel.eval_rob_accuracy_gn(test_loader)  # gaussian noise accuracy

In [ ]:
rmodel.eval_rob_accuracy_fgsm(test_loader, eps)  # FGSM accuracy

In [ ]:
rmodel.eval_rob_accuracy_pgd(test_loader, eps, alpha, steps)  # PGD accuracy